Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 10
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [3]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end

function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s])
    end
    return as
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

cost (generic function with 2 methods)

## Form library randomly

In [7]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 100000)
spud_i = 0
ss_prob = 1.1

1.1

In [8]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [9]:
library = unique(library[1:spud_i])
n_spuds = length(library)

12675

In [55]:
function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end


ffp (generic function with 1 method)

In [60]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env];

In [61]:
counts = ffp(nash_env[1:15], 10000)

15-element Vector{Int64}:
    1
 1334
    1
 1879
  925
    2
 1146
  153
    1
  263
 1326
 1302
    1
 1680
    1

In [62]:
nash_env = nash_env[1:15][counts .> 10]

9-element Vector{Spud}:
 Spud("c4458137", 3, 2, 10, 10, 10, 10)
 Spud("c3597175", 4, 2, 10, 10, 10, 6)
 Spud("c1595416", 7, 1, 10, 10, 3, 10)
 Spud("c1267092", 3, 9, 3, 3, 3, 1)
 Spud("c1265702", 2, 3, 10, 10, 10, 4)
 Spud("c1228093", 8, 7, 1, 1, 2, 2)
 Spud("c1195209", 3, 7, 3, 5, 1, 4)
 Spud("c1158556", 10, 1, 8, 4, 1, 5)
 Spud("c1089004", 10, 4, 4, 1, 1, 3)

In [64]:
n_nash = length(nash_env)

9

In [148]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [149]:
payoffs = get_payoffs(nash_env)

9×9 Matrix{Int64}:
  0   1   1  -1   1  -1  -1   1  -1
 -1   0   1  -1   1  -1  -1   1   1
 -1  -1   0  -1  -1   1   1   1   1
  1   1   1   0  -1   1  -1  -1  -1
 -1  -1   1   1   0  -1   1   1  -1
  1   1  -1  -1   1   0   1  -1  -1
  1   1  -1   1  -1  -1   0  -1  -1
 -1  -1  -1   1  -1   1   1   0   1
  1  -1  -1   1   1   1   1  -1   0

## Attempt to use Gaussian elimination

In [67]:
#var_order = reverse(1:n_nash)
var_order = 1:n_nash
transformed_rows = copy(payoffs);
row_coeffs = Array{Int64}(undef, (n_nash, n_nash)) .* 0;
for i in 1:n_nash
    row_coeffs[i, i] = 1
end
var_i = 1
add_row = rand(findall(transformed_rows[:, var_order[var_i]] .!= 0))
cf_new = sign(transformed_rows[add_row, var_order[var_i]])
old_row = transformed_rows[var_i, :]
new_row = cf_new .* transformed_rows[add_row, :]
old_coeff = row_coeffs[var_i, :]
new_coeff = [0 for i in 1:n_nash];
new_coeff[add_row] = cf_new;
transformed_rows[add_row, :] = old_row
transformed_rows[var_i, :] = new_row
row_coeffs[add_row, :] = old_coeff
row_coeffs[var_i, :] = new_coeff;
for i in (var_i + 1):n_nash
    cf0 = transformed_rows[i, var_order[var_i]]
    cf1 = transformed_rows[var_i, var_order[var_i]]
    gc = gcd(cf0, cf1)
    cf1 = div(cf1,gc)
    cf0 = div(cf0,gc)
    transformed_rows[i, :] = cf1 .* transformed_rows[i, :] .- cf0 .* transformed_rows[var_i, :]
    row_coeffs[i, :] = cf1 .* row_coeffs[i, :] .- cf0 .* row_coeffs[var_i, :]
end

In [68]:
for var_i in 2:n_nash
    println(var_i)
    cands = findall(transformed_rows[:, var_order[var_i]] .!= 0)
    cands = cands[cands .>= var_i]
    if length(cands) > 0
        avs = map(abs, transformed_rows[cands, var_order[var_i]])
        cands = cands[avs .== minimum(avs)]
        add_row = rand(cands)
        new_row = transformed_rows[add_row, :]
        new_coeff = row_coeffs[add_row, :]
        old_row = transformed_rows[var_i, :]
        old_coeff = row_coeffs[var_i, :]        
        cf_new = sign(new_row[var_order[var_i]])
        new_row = cf_new .* new_row
        new_coeff = cf_new .* new_coeff
        for i in 1:(var_i - 1)
            cf0 = new_row[var_order[i]]
            cf1 = transformed_rows[i, var_order[i]]
            # check what multiplier is needed
            gc = gcd(cf0, cf1)
            cf1 = div(cf1,gc)
            cf0 = div(cf0,gc)
            new_row = cf1 .* new_row .- cf0 .* transformed_rows[i, :]
            new_coeff = cf1 .* new_coeff
            new_coeff = new_coeff - cf0 .* row_coeffs[i, :]
        end
        transformed_rows[add_row, :] = old_row
        transformed_rows[var_i, :] = new_row
        row_coeffs[add_row, :] = old_coeff
        row_coeffs[var_i, :] = new_coeff
    end
    for i in (var_i + 1):n_nash
        cf0 = transformed_rows[i, var_order[var_i]]
        cf1 = transformed_rows[var_i, var_order[var_i]]
        gc = gcd(cf0, cf1)
        cf1 = div(cf1,gc)
        cf0 = div(cf0,gc)
        transformed_rows[i, :] = cf1 .* transformed_rows[i, :] .- cf0 .* transformed_rows[var_i, :]
        row_coeffs[i, :] = cf1 .* row_coeffs[i, :] .- cf0 .* row_coeffs[var_i, :]
    end
end

2
3
4
5
6
7
8
9


In [69]:
transformed_rows

9×9 Matrix{Int64}:
 1  1  -1  -1  0   1  -1  -1   1
 0  1   0  -2  1   0  -2   0   2
 0  0   1   2  1  -2   0   0  -2
 0  0   0   1  1  -1  -1  -1   1
 0  0   0   0  1  -1   2   0  -2
 0  0   0   0  0   1  -3  -3   5
 0  0   0   0  0   0   5  -7   0
 0  0   0   0  0   0   0   7  -5
 0  0   0   0  0   0   0   0   0

In [71]:
a = [0 for i in 1:n_nash]
for i in 1:n_nash
    a = a .+ payoffs[i, :] .* row_coeffs[8, i]
end
maximum(a .- transformed_rows[8, :])

0

In [72]:
minimum([transformed_rows[i,var_order[i]] for i in 1:n_nash])

0

In [20]:
# invertible because it includes dominated strats

## Using Power iterations

In [73]:
#nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
#nash_env = df_to_spuds(nash_env_df)
#counts = [parse(Int, s.name[2:end]) for s in nash_env];
#n_nash = length(nash_env)
#payoffs = Array{Int64}(undef, (n_nash, n_nash))
# for i in 1:n_nash
#     for j in 1:n_nash
#         payoffs[i, j] = eval_battle(nash_env[i], nash_env[j])
#     end
# end

In [118]:
function power_iterate(payoffs::Array{Int64}, maxv::Int64, nits::Int64)::Array{Int64}
    n_nash = size(payoffs)[1]
    payoffs2 = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs2[i, j] = sum(payoffs[i, :] .* payoffs[:, j])
        end
    end
    v = [0 for i in 1:n_nash]
    for i in 1:20
        v[rand(1:n_nash)] = 1
        v[rand(1:n_nash)] = -1
    end
    v = map(x -> div(x * 1000, maximum(map(abs, v))), v)
    oldnorm = sum(v .* v)
    newnorm = 0
    for i in 1:20
        v2 = [sum(-payoffs2[i, :] .* v) for i in 1:n_nash]
        v2
        v = v2
        v[1:3]
        newnorm = sum(v .* v)
        #println(newnorm/oldnorm)
        v = map(x -> div(x * 1000, maximum(map(abs, v))), v)
        oldnorm = sum(v .* v)
    end
    ratio = newnorm/oldnorm
    mult = 2 * sqrt(sqrt(ratio))
    mult = convert(Int64, floor(mult))
    v = [maxv for i in 1:n_nash]; 
    for it_count in 1:nits
        #if mod(it_count, 10000) == 0
        #    println(minimum(v))
        #end
        v2 = [sum(payoffs2[i, :] .* v) for i in 1:n_nash];
        v = v .* mult .+ map(x -> div(x, mult), v2);
        #println(log(maximum(v))/log(2))
        v = map(x -> div(x, maximum(v)), v .* maxv);
        v[v .< 0] .= 0
    end
    return v
end

power_iterate (generic function with 1 method)

In [142]:
@time v = power_iterate(payoffs, 1000000, 10000)
w_power = v./sum(v);

  0.228164 seconds (1.07 M allocations: 50.058 MiB, 5.30% gc time)


In [143]:
maximum([sum(payoffs[i, :] .* w_power) for i in 1:n_nash])

1.9801548481079845e-5

In [136]:
@time counts = ffp(nash_env, 2000000)
w_ffp = counts ./ sum(counts);

  0.444364 seconds (8.00 M allocations: 549.471 MiB, 20.65% gc time)


In [137]:
maximum([sum(payoffs[i, :] .* w_ffp) for i in 1:n_nash])

0.0011144949847725905

## Using Linear programming (rock-paper-scissors)

In [144]:
using JuMP
using HiGHS

In [276]:
vector_model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [277]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds(nash_env_df)[1:15]

15-element Vector{Spud}:
 Spud("c12207143", 6, 1, 10, 10, 10, 10)
 Spud("c4458137", 3, 2, 10, 10, 10, 10)
 Spud("c3701536", 9, 7, 1, 2, 1, 1)
 Spud("c3597175", 4, 2, 10, 10, 10, 6)
 Spud("c1595416", 7, 1, 10, 10, 3, 10)
 Spud("c1400790", 10, 4, 3, 2, 1, 4)
 Spud("c1267092", 3, 9, 3, 3, 3, 1)
 Spud("c1265702", 2, 3, 10, 10, 10, 4)
 Spud("c1244219", 6, 8, 2, 1, 3, 1)
 Spud("c1228093", 8, 7, 1, 1, 2, 2)
 Spud("c1195209", 3, 7, 3, 5, 1, 4)
 Spud("c1158556", 10, 1, 8, 4, 1, 5)
 Spud("c1137473", 2, 10, 10, 2, 1, 3)
 Spud("c1089004", 10, 4, 4, 1, 1, 3)
 Spud("c1059805", 7, 1, 10, 10, 10, 3)

In [278]:
payoffs = get_payoffs(nash_env)

15×15 Matrix{Int64}:
  0  -1  -1  -1   1   1  -1  -1  -1  -1  -1   1  -1   1   1
  1   0  -1   1   1  -1  -1   1  -1  -1  -1   1   1  -1   1
  1   1   0   1  -1  -1  -1   1   1  -1   1  -1  -1  -1  -1
  1  -1  -1   0   1   1  -1   1  -1  -1  -1   1   1   1   1
 -1  -1   1  -1   0   1  -1  -1  -1   1   1   1  -1   1  -1
 -1   1   1  -1  -1   0   1   1   1   1   1  -1   1  -1  -1
  1   1   1   1   1  -1   0  -1   1   1  -1  -1   1  -1   1
  1  -1  -1  -1   1  -1   1   0  -1  -1   1   1   1  -1   1
  1   1  -1   1   1  -1  -1   1   0  -1  -1  -1  -1  -1   1
  1   1   1   1  -1  -1  -1   1   1   0   1  -1  -1  -1  -1
  1   1  -1   1  -1  -1   1  -1   1  -1   0  -1   1  -1   1
 -1  -1   1  -1  -1   1   1  -1   1   1   1   0  -1   1  -1
  1  -1   1  -1   1  -1  -1  -1   1   1  -1   1   0   1   1
 -1   1   1  -1  -1   1   1   1   1   1   1  -1  -1   0  -1
 -1  -1   1  -1   1   1  -1  -1  -1   1  -1   1  -1   1   0

In [279]:
#payoffs = hcat([0; 1; -1], [-1; 0; 1], [1; -1; 0])

In [280]:
n_nash = size(payoffs)[1]
# min <c, x> s.t. Ax = b, x>= 0


15

```
variable 4-6 will be the value of the game
Ax[1:3] + x[4:6] - x[7] = -1

[A  I   -1] x[1:3] =  [1]
[1  0   0 ] x[4:6]  = [1]
            x[7]

also 1'x = 1

Ax = 

x[7] (= 1 + value of game) to be minimized
```

In [281]:
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv)

16×31 Matrix{Int64}:
  0  -1  -1  -1   1   1  -1  -1  -1  …  0  0  0  0  0  0  0  0  0  0  0  -1
  1   0  -1   1   1  -1  -1   1  -1     0  0  0  0  0  0  0  0  0  0  0  -1
  1   1   0   1  -1  -1  -1   1   1     0  0  0  0  0  0  0  0  0  0  0  -1
  1  -1  -1   0   1   1  -1   1  -1     0  0  0  0  0  0  0  0  0  0  0  -1
 -1  -1   1  -1   0   1  -1  -1  -1     1  0  0  0  0  0  0  0  0  0  0  -1
 -1   1   1  -1  -1   0   1   1   1  …  0  1  0  0  0  0  0  0  0  0  0  -1
  1   1   1   1   1  -1   0  -1   1     0  0  1  0  0  0  0  0  0  0  0  -1
  1  -1  -1  -1   1  -1   1   0  -1     0  0  0  1  0  0  0  0  0  0  0  -1
  1   1  -1   1   1  -1  -1   1   0     0  0  0  0  1  0  0  0  0  0  0  -1
  1   1   1   1  -1  -1  -1   1   1     0  0  0  0  0  1  0  0  0  0  0  -1
  1   1  -1   1  -1  -1   1  -1   1  …  0  0  0  0  0  0  1  0  0  0  0  -1
 -1  -1   1  -1  -1   1   1  -1   1     0  0  0  0  0  0  0  1  0  0  0  -1
  1  -1   1  -1   1  -1  -1  -1   1     0  0  0  0  0  0  0  0  1  

In [282]:
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
transpose(b)

1×16 transpose(::Vector{Int64}) with eltype Int64:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  1

In [283]:
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;

In [284]:
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);

In [285]:
@constraint(vector_model, A * x .== b)

16-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -x[2] - x[3] - x[4] + x[5] + x[6] - x[7] - x[8] - x[9] - x[10] - x[11] + x[12] - x[13] + x[14] + x[15] + x[16] - x[31] = -1
 x[1] - x[3] + x[4] + x[5] - x[6] - x[7] + x[8] - x[9] - x[10] - x[11] + x[12] + x[13] - x[14] + x[15] + x[17] - x[31] = -1
 x[1] + x[2] + x[4] - x[5] - x[6] - x[7] + x[8] + x[9] - x[10] + x[11] - x[12] - x[13] - x[14] - x[15] + x[18] - x[31] = -1
 x[1] - x[2] - x[3] + x[5] + x[6] - x[7] + x[8] - x[9] - x[10] - x[11] + x[12] + x[13] + x[14] + x[15] + x[19] - x[31] = -1
 -x[1] - x[2] + x[3] - x[4] + x[6] - x[7] - x[8] - x[9] + x[10] + x[11] + x[12] - x[13] + x[14] - x[15] + x[20] - x[31] = -1
 -x[1] + x[2] + x[3] - x[4] - x[5] + x[7] + x[8] + x[9] + x[10] + x[11] - x[12] + x[13] - x[14] - x[15] + x[21] - x[31] = -1
 x[1] + x[2] + x[3] + x[4] + x[5] - x[6] - x[8] + x[9] + x[10] - x[11] - x[12] + x

In [286]:
@objective(vector_model, Min, c' * x)

x[31]

In [287]:
vector_model

A JuMP Model
Minimization problem with:
Variables: 31
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 16 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 31 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [292]:
@time optimize!(vector_model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  1.0000000000e+00
HiGHS run time      :          0.00
  0.000109 seconds (2 allocations: 32 bytes)


In [293]:
[value(x[i]) for i in 1:(2 * n_nash + 1)];

In [294]:
w_lp = [value(x[i]) for i in 1:n_nash]
sum(w_lp)

1.0000000000000018

In [295]:
w_lp = w_lp./sum(w_lp);
maximum([sum(payoffs[i, :] .* w_lp) for i in 1:n_nash])

2.525757381022231e-15